In [1]:
import pandas as pd
import numpy as np

In [2]:
from keras.models import Model 
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D 
from keras.utils import np_utils

E:\Ivan\Anaconda\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
mnist_train = pd.read_csv('train.csv')
mnist_test = pd.read_csv('test.csv')

In [4]:
batch_size = 128 
num_epochs = 30

num_train = mnist_train.shape[0]
num_test = mnist_test.shape[0]

height, width, depth = 28, 28, 1 
input_shape = (height, width, depth)
num_classes = 10 

X_train, X_test, y_train = mnist_train.drop(['label'],axis=1).values, mnist_test.values, mnist_train['label'].values

X_train = X_train.reshape(num_train, height, width, depth) 
X_test = X_test.reshape(num_test, height, width, depth)
X_train = X_train.astype('float32') 
X_test = X_test.astype('float32')
X_train /= 255 
X_test /= 255 

Y_train = np_utils.to_categorical(y_train, num_classes)

In [5]:
inp = Input(shape = input_shape)
conv_1  = Conv2D(32, (3, 3), activation='relu')(inp)
conv_2 = Conv2D(64, (3, 3), activation='relu')(conv_1)
pool = MaxPooling2D(pool_size=(2, 2))(conv_2)
drop_1 = Dropout(0.25)(pool)
flat = Flatten()(drop_1)
hidden = Dense(128, activation='relu')(flat)
drop_2 = Dropout(0.5)(hidden)
out = Dense(num_classes, activation='softmax')(drop_2)
model = Model(input=inp, output=out)

E:\Ivan\Anaconda\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  # Remove the CWD from sys.path while we load stuff.


In [6]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=num_epochs, verbose=1) 


E:\Ivan\Anaconda\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/30
42000/42000 [==============================] - 89s 2ms/step - loss: 0.2978 - acc: 0.9069
Epoch 2/30
42000/42000 [==============================] - 89s 2ms/step - loss: 0.0950 - acc: 0.9716
Epoch 3/30
42000/42000 [==============================] - 89s 2ms/step - loss: 0.0709 - acc: 0.9779
Epoch 4/30
42000/42000 [==============================] - 89s 2ms/step - loss: 0.0573 - acc: 0.9832
Epoch 5/30
42000/42000 [==============================] - 89s 2ms/step - loss: 0.0478 - acc: 0.9854
Epoch 6/30
42000/42000 [==============================] - 89s 2ms/step - loss: 0.0431 - acc: 0.9868
Epoch 7/30
42000/42000 [==============================] - 89s 2ms/step - loss: 0.0388 - acc: 0.9870
Epoch 8/30
42000/42000 [==============================] - 89s 2ms/step - loss: 0.0313 - acc: 0.9893
Epoch 9/30
42000/42000 [==============================] - 89s 2ms/step - loss: 0.0283 - acc: 0.9908
Epoch 10/30
42000/42000 [==============================] - 89s 2ms/step - loss: 0.0276 - acc: 0.9911

In [8]:
test_predict = model.predict(X_test)

In [13]:
y_predict = []
for predict in test_predict:
    max_value = 0
    index_max_value = 0
    i = 0
    for val in predict:
        if max_value < val:
            max_value = val
            index_max_value = i
        i += 1
    y_predict.append(index_max_value)

In [15]:
rezult = pd.DataFrame({'ImageId': range(1,28001), 'Label': np.array(y_predict)})
rezult.to_csv('submission.csv', sep=',', index=False)